In [4]:
import pytest
import sympy
import underworld3 as uw

# These are tested by test_001_meshes.py

structured_quad_box = uw.meshing.StructuredQuadBox(elementRes=(3,) * 2)

unstructured_quad_box_irregular = uw.meshing.UnstructuredSimplexBox(
    cellSize=0.2, regular=False, qdegree=2, refinement=1
)
unstructured_quad_box_regular = uw.meshing.UnstructuredSimplexBox(
    cellSize=0.2, regular=True, qdegree=2, refinement=2
)

unstructured_quad_box_irregular_3D = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0, 0.0),
    maxCoords=(1.0, 1.0, 1.0),
    cellSize=0.25,
    regular=False,
    qdegree=2,
)

# Maybe lower and upper would work better for the names of the box mesh boundaries too.


def test_stokes_boxmesh(mesh):
    print(f"Mesh - Coordinates: {mesh.CoordinateSystem.type}")
    mesh.dm.view()

    if mesh.dim == 2:
        x, y = mesh.X
    else:
        x, y, z = mesh.X

    u = uw.discretisation.MeshVariable(
        r"U", mesh, mesh.dim, vtype=uw.VarType.VECTOR, degree=2, varsymbol=R"{\mathbf{u}}",
    )
    p = uw.discretisation.MeshVariable(
        r"P", mesh, 1, vtype=uw.VarType.SCALAR, degree=1,  varsymbol=R"{\mathbf{p}}",
    )

    stokes = uw.systems.Stokes(mesh, velocityField=u, pressureField=p)
    stokes.constitutive_model = uw.constitutive_models.ViscoElasticPlasticFlowModel
    stokes.constitutive_model.Parameters.shear_viscosity_0 = 1
    stokes.constitutive_model.Parameters.shear_modulus = 1
    stokes.constitutive_model.Parameters.dt_elastic = sympy.sympify(1) / 10
    

    stokes.petsc_options["snes_type"] = "newtonls"
    stokes.petsc_options["ksp_type"] = "fgmres"

    stokes.petsc_options["snes_type"] = "newtonls"
    stokes.petsc_options["ksp_type"] = "fgmres"
    stokes.petsc_options["ksp_monitor"] = None
    stokes.petsc_options["snes_monitor"] = None
    stokes.tolerance = 1.0e-3

    # stokes.petsc_options.setValue("fieldsplit_velocity_pc_type", "mg")
    stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_type", "kaskade")
    stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_cycle_type", "w")

    stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
    stokes.petsc_options[f"fieldsplit_velocity_ksp_type"] = "fcg"
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_type"] = "chebyshev"
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_max_it"] = 7
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_converged_maxits"] = None

    stokes.petsc_options.setValue("fieldsplit_pressure_pc_type", "gamg")
    stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_type", "multiplicative")
    stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_cycle_type", "v")

    if mesh.dim == 2:
        stokes.bodyforce = 1.0e6 * sympy.Matrix([0, x])

        stokes.add_dirichlet_bc((0.0, 0.0), "Bottom")
        stokes.add_dirichlet_bc((0.0, 0.0), "Top", 0)

        stokes.add_dirichlet_bc((0.0, sympy.oo), "Left")
        stokes.add_dirichlet_bc((0.0, sympy.oo), "Right")
    else:
        stokes.bodyforce = 1.0e6 * sympy.Matrix([0, x, 0])

        stokes.add_dirichlet_bc((0.0, 0.0, 0.0), "Bottom")
        stokes.add_dirichlet_bc((0.0, 0.0, 0.0), "Top")

        stokes.add_dirichlet_bc((0.0, sympy.oo, sympy.oo), "Left")
        stokes.add_dirichlet_bc((0.0, sympy.oo, sympy.oo), "Right")

        stokes.add_dirichlet_bc((sympy.oo, 0.0, sympy.oo), "Front")
        stokes.add_dirichlet_bc((sympy.oo, 0.0, sympy.oo), "Back")

    stokes.solve(verbose=True)

    print(f"Mesh dimensions {mesh.dim}", flush=True)
    stokes.view()

    assert stokes.snes.getConvergedReason() > 0


    return stokes.constitutive_model.viscosity


## Note this one fails because the corner boundary condition is not applied
## correctly when the regular simplex mesh is used.
## Mark as xfail for now


def test_stokes_boxmesh_bc_failure(mesh):
    # mesh = unstructured_quad_box_regular

    print(f"Mesh - Coordinates: {mesh.CoordinateSystem.type}")
    mesh.dm.view()

    if mesh.dim == 2:
        x, y = mesh.X
    else:
        x, y, z = mesh.X

    u = uw.discretisation.MeshVariable(
        r"mathbf{u}", mesh, mesh.dim, vtype=uw.VarType.VECTOR, degree=2
    )
    p = uw.discretisation.MeshVariable(
        r"mathbf{p}", mesh, 1, vtype=uw.VarType.SCALAR, degree=1
    )

    stokes = uw.systems.Stokes(mesh, velocityField=u, pressureField=p)
    stokes.constitutive_model = uw.constitutive_models.ViscoElasticPlasticFlowModel
    stokes.constitutive_model.Parameters.shear_viscosity_0 = 1

    stokes.petsc_options["snes_type"] = "newtonls"
    stokes.petsc_options["ksp_type"] = "fgmres"

    stokes.petsc_options["snes_monitor"] = None
    stokes.petsc_options["ksp_monitor"] = None

    # stokes.petsc_options.setValue("fieldsplit_velocity_pc_type", "mg")
    stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_type", "kaskade")
    stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_cycle_type", "w")

    stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
    stokes.petsc_options[f"fieldsplit_velocity_ksp_type"] = "fcg"
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_type"] = "chebyshev"
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_max_it"] = 7
    stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_converged_maxits"] = None

    stokes.petsc_options.setValue("fieldsplit_pressure_pc_type", "gamg")
    stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_type", "additive")
    stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_cycle_type", "v")

    if mesh.dim == 2:
        stokes.bodyforce = 1.0e6 * sympy.Matrix([0, x])

        stokes.add_dirichlet_bc((0.0, 0.0), "Bottom")
        stokes.add_dirichlet_bc((0.0, 0.0), "Top", 0)

        stokes.add_dirichlet_bc((0.0, sympy.oo), "Left")
        stokes.add_dirichlet_bc((0.0, sympy.oo), "Right")
    else:
        stokes.bodyforce = 1.0e6 * sympy.Matrix([0, x, 0])

        stokes.add_dirichlet_bc((0.0, 0.0, 0.0), "Bottom")
        stokes.add_dirichlet_bc((0.0, 0.0, 0.0), "Top")

        stokes.add_dirichlet_bc((0.0, sympy.oo, sympy.oo), "Left")
        stokes.add_dirichlet_bc((0.0, sympy.oo, sympy.oo), "Right")

        stokes.add_dirichlet_bc((sympy.oo, 0.0, sympy.oo), "Front")
        stokes.add_dirichlet_bc((sympy.oo, 0.0, sympy.oo), "Back")

    stokes.solve()

    print(f"Mesh dimensions {mesh.dim}", flush=True)
    stokes.dm.ds.view()

    assert stokes.snes.getConvergedReason() > 0

    return stokes.stokes.constitutive_model.viscosity



In [6]:
sympy.core.cache.clear_cache()

In [5]:
eta_eff = test_stokes_boxmesh(structured_quad_box)

Mesh - Coordinates: Cartesian 2D
DM Object: uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(1.0, 1.0).msh 1 MPI process
  type: plex
uw_.meshes/uw_structuredQuadBox_minC(0.0, 0.0)_maxC(1.0, 1.0).msh in 2 dimensions:
  Number of 0-cells per rank: 16
  Number of 1-cells per rank: 24
  Number of 2-cells per rank: 9
Labels:
  depth: 3 strata with value/size (0 (16), 1 (24), 2 (9))
  All_Boundaries: 1 strata with value/size (1001 (12))
  Bottom: 1 strata with value/size (11 (5))
  Elements: 1 strata with value/size (99999 (13))
  Left: 1 strata with value/size (14 (5))
  Right: 1 strata with value/size (13 (5))
  Top: 1 strata with value/size (12 (5))
  celltype: 3 strata with value/size (0 (16), 1 (24), 4 (9))
  UW_Boundaries: 5 strata with value/size (11 (5), 12 (5), 13 (5), 14 (5), 1001 (12))
Processing JIT    0 / Matrix([[0, -1000000.0*N.x]])
Processing JIT    1 / Matrix([[-{ { \,\!\,\!\,\!\,\!\,\! } {\mathbf{p}} }(N.x, N.y) + 2*{ { \,\!\,\!\,\!\,\!\,\! } {\mathbf{u}} }_{ 0,0}(N.x, 

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

**Saddle point system solver**

Primary problem: 

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

This solver is formulated in 2 dimensions

In [ ]:
eta_eff

In [ ]:
eta_eff.sub_all(keep_constants=False)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="Pressure", ylabel="Depth")
ax1.plot(pressure_interp, ycoords, linewidth=3, label="Numerical solution")
ax1.plot(pressure_interp_0, ycoords, linewidth=3, label="Numerical solution (no G)")
ax1.plot(
    pressure_analytic, ycoords, linewidth=3, linestyle="--", label="Analytic solution"
)

ax1.plot(
    pressure_analytic_noG,
    ycoords,
    linewidth=3,
    linestyle="--",
    label="Analytic (no gravity)",
)

ax1.grid("on")
ax1.legend()

In [ ]:
pressure_interp, pressure_analytic, pressure_interp_0, pressure_analytic_noG, ycoords =  test_Darcy_boxmesh_G_and_noG(meshSimplex_box_irregular)



In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="Pressure", ylabel="Depth")
ax1.plot(pressure_interp, ycoords, linewidth=3, label="Numerical solution")
ax1.plot(pressure_interp_0, ycoords, linewidth=3, label="Numerical solution (no G)")
ax1.plot(
    pressure_analytic, ycoords, linewidth=3, linestyle="--", label="Analytic solution"
)

ax1.plot(
    pressure_analytic_noG,
    ycoords,
    linewidth=3,
    linestyle="--",
    label="Analytic (no gravity)",
)

ax1.grid("on")
ax1.legend()

In [ ]:
sympy.core.cache.